In [1]:
import sympy as sp
import numpy as np

In [2]:
from deepgroebner.binomials import BinomialBuchbergerEnv, BinomialLeadMonomialsWrapper, to_binomial, reduce
from deepgroebner.buchberger import BuchbergerEnv, LeadMonomialsWrapper
from deepgroebner.ideals import RandomBinomialIdealGenerator

## Check `reduce`

In [3]:
ideal_gen = RandomBinomialIdealGenerator(3, 5, 3)

In [4]:
for i in range(1000):
    F = next(ideal_gen)
    FB = [to_binomial(f) for f in F]
    B1 = to_binomial(F[0].rem(F[1:]))
    B2, _ = reduce(FB[0], FB[1:])
    assert B1 == B2

## Check environments

In [5]:
ideal_gen = RandomBinomialIdealGenerator(3, 5, 3)

In [6]:
count = 0

for i in range(1000):
    
    env1 = BinomialBuchbergerEnv(ideal_gen)
    env1 = BinomialLeadMonomialsWrapper(env1, k=2)
    np.random.seed(i)
    state1 = env1.reset()

    env2 = BuchbergerEnv(ideal_gen)
    env2 = LeadMonomialsWrapper(env2, k=2)
    np.random.seed(i)
    state2 = env2.reset()

    done1, done2 = False, False
    while not done1 and not done2:
        state1, reward1, done1, _ = env1.step(0)
        state2, reward2, done2, _ = env2.step(0)
        assert np.array_equal(state1, state2)
        if not reward1 == reward2:
            count += 1
        assert done1 == done2

count

1018

## Compare times

In [7]:
ideal_gen = RandomBinomialIdealGenerator(5, 10, 5)
seed = 12

In [8]:
env = BinomialBuchbergerEnv(ideal_gen)
env = BinomialLeadMonomialsWrapper(env, k=2)
np.random.seed(seed)

In [9]:
%%time

steps = 0

for i in range(10):
    state = env.reset()
    done = False
    while not done:
        action = np.random.randint(state.shape[0])
        state, reward, done, info = env.step(action)
        steps += 1

steps

CPU times: user 7.02 s, sys: 16.3 ms, total: 7.04 s
Wall time: 7.03 s


3256

In [10]:
env = BuchbergerEnv(ideal_gen)
env = LeadMonomialsWrapper(env, k=2)
np.random.seed(seed)

In [11]:
%%time

steps = 0

for i in range(10):
    state = env.reset()
    done = False
    while not done:
        action = np.random.randint(state.shape[0])
        state, reward, done, info = env.step(action)
        steps += 1

steps

CPU times: user 3.47 s, sys: 18 ms, total: 3.48 s
Wall time: 3.49 s


3256